# Grupo 03 - Desafío 01
## Estrategia de construcción
>**OBJETIVO:** Crear un df de post-producción que luego se transformará en producción para realizar análisis del dataset.
> * Se trabajará depurando cada columna del dataset con la finalidad de reconstruir datos perdidos y/o crear nuevas columnas que serán utilizadas a posteriori en la fase de análisis.

### Instalación de librerías

In [ ]:
#!pip install unidecode

### Importación de librerías

In [2]:
import sys
sys.path.append("../src")
import g3utils as g3

import pandas as pd
import numpy as np
import re as re
import unidecode

# Carga e inicialización

In [21]:
# Definición de filas a cargar en las pruebas

#ca_filas = 40
ca_filas = 121220

In [4]:
# Carga del dataset en un df

df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))
#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [5]:
# Reemplado de NaN's

df = g3.reemplaza_nan('sin datos',df)

In [6]:
# Verificación del reemplazo

df.shape

(121220, 27)

## Armado inicial de df post-produccion
* Este bloque se reserva para realizar la precarga inicial dentro de un df de post-producción que finalmente se transformará en dataset de producción.

In [22]:
# Dadtaframe de posproducción. Tras depurar todas las regex, este DF será la base de análisis de negocio
# Se ejecuta una única vez y se le van incorporando las columnas depuradas.
# Tener en cuenta que el DF pospo definitivo tiene que tener la misma cantidad de filas que el DF base len(df)

pospo = g3.generar_df_posproduccion(ca_filas)
pospo.shape
#pospo

(121220, 1)

## Procesamiento de limpieza y carga de columnas depuradas en df post-produccion 'pospo'
> *** Modo de uso:** 
>* En este markdown se vuelcan las funciones que generan y agregan columnas depuradas dentro de pospo
>* La codificación de desarrollo se realiza mas abajo dentro del título **Desarrollo** y una vez terminado se empaqueta dentro de una función consolidadora.



In [23]:
# Depuración de expresion 'ambientes' en 'description' y generacion de columna en pospro

def recuperar_ambientes(pospo):
    pattern = r'(\d*\w*)\s*amb'  # Patron para buscar metros ambientes
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas))
    dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
    df_temp = g3.limpiar_columna_x_clave(dic, df1)
    pospo = g3.agregar_columna('r_ambiente', pospo, df_temp)
    return pospo


In [24]:
pospo = recuperar_ambientes(pospo)
pospo.shape

(121220, 2)

In [25]:
# Depuración 'precio usd' en 'description' y agregado de columna en pospro

def recuperar_usd(pospo):
    pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?u[$sd]\w?|u[$sd]\w?\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio_usd
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado(df1, 'r_usd') # Genera DF acotado con los valores recuperados unicamente
    pospo = pd.merge(pospo, df2,on='indice', how='left')
    return pospo


In [26]:
pospo=recuperar_usd(pospo)
pospo.shape


(121220, 3)

In [27]:
def recuperar_metros(pospo):
    pattern = r"(\d*)\,?\d*\s?(?:m²|mts\s?2|metros\s?2|mts²|m2|metros\s?cuadrado|mts\s?cuadrado)" # superficie
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_metros2') # Genera DF acotado con los valores recuperados unicamente
    pospo = pd.merge(pospo, df2,on='indice', how='left')
    return pospo

In [28]:
pospo=recuperar_metros(pospo)
pospo.shape

(121220, 4)

In [29]:
def recuperar_frente(pospo):
    pattern = r"(frente)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_frente')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_frente')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo

In [30]:
pospo=recuperar_frente(pospo)
pospo.shape

(121220, 5)

In [31]:
def recuperar_contrafrente(pospo):
    pattern = r"(contrafrente)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_contrafrente')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_contrafrente')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo

In [32]:
pospo=recuperar_contrafrente(pospo)
pospo.shape

(121220, 6)

In [33]:
# Se verifico previamente un nro considerable de valores recuperados en description y title por esa razón se incluyeron

def recuperar_expensas(pospo):
    pattern = r"(expensas)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_expensas')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_expensas')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo


In [34]:
pospo=recuperar_expensas(pospo)
pospo.shape

(121220, 7)

In [35]:
pospo

,indice,r_ambiente,r_usd,r_metros2,r_frente,r_contrafrente,r_expensas
0,0,2.0,NaN,NaN,NaN,NaN,expensas
1,1,NaN,20.000,NaN,frente,NaN,NaN
2,2,2.0,NaN,NaN,NaN,NaN,expensas
3,3,3.0,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,frente,contrafrente,NaN
5,5,NaN,NaN,NaN,frente,NaN,NaN
6,6,NaN,NaN,106,frente,NaN,expensas
7,7,1.0,NaN,NaN,NaN,NaN,NaN
8,8,2.0,NaN,NaN,NaN,NaN,NaN
9,9,NaN,NaN,NaN,NaN,NaN,NaN


# Desarrollo